In [ ]:
from nltk.translate.nist_score import sentence_nist
import os
import nltk
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
def cac_avg(l):
    s = 0
    for i in l:
        s+=float(i)
    return s/len(l)
    
def cal_nist(references, candidate):
    references = [" ".join(tokenizer.tokenize(i)) for i in references]
    candidate = " ".join(tokenizer.tokenize(candidate))

    refs = []
    for ref in references:
        t = []
        sents  = nltk.sent_tokenize(ref)
        words = []
        for sent in sents:
            words.extend(nltk.word_tokenize(sent))
        refs.append(words)
    sents = nltk.sent_tokenize(candidate)
    cands = []
    for sent in sents:
        cands.extend(nltk.word_tokenize(sent))
    try:
        score=sentence_nist(refs, cands,)
    except ZeroDivisionError as ze:
        score = 0
    return score
import spacy
nlp =spacy.load('en_core_web_sm')
def cut(s):
    doc = nlp(s)
    tokens = [i.text for i in doc]
    return " ".join(tokens)

In [ ]:
refs = []
with open("./references/CE/1.CE ref.txt", encoding="utf8") as f:
    refs.append(f.readlines())
with open("./references/CE/2.CE ref.txt", encoding="utf8") as f:
    refs.append(f.readlines())
with open("./references/CE/3.CE ref.txt", encoding="utf8") as f:
    refs.append(f.readlines())
with open("./references/CE/4.CE ref.txt", encoding="utf8") as f:
    refs.append(f.readlines())

p_refs = []
n=0
for ref in refs:
    t = []
    n+=1
    for s in ref:
        if len(s)>5:
            a=s[s.index(".")+1:].strip().lower()
            t.append(cut(a))
    p_refs.append(t)
refs = p_refs
  
file_path = "./C-E transcripts-sentence-txt/"

files = []
for i,j,k in os.walk(file_path):
    files.extend(k)


In [ ]:
nist_dict = {}
for fn in files:
    with open(file_path+fn, "r", encoding="utf8") as f:
        print(fn)
        pgs = f.readlines()
        key = fn.split(".")[0]
        nist_dict[key] = {"nist_val":[],"avg_nist":0}           
        for i in range(len(pgs)):
            p_refs =[] 
            for j in range(len(refs)):             
                p_refs.append(refs[j][i])
            nist_scores = cal_nist(p_refs, pgs[i])
            nist_dict[key]["nist_val"].append(nist_scores)
        nist_dict[key]["avg_nist"] = cac_avg(nist_dict[key]["nist_val"])
import json
with open("ce_nist_score.json", "w", encoding="utf8") as f:
    json.dump(nist_dict, f, indent=4,sort_keys=True)
print(nist_dict)  

In [ ]:
with open("ce_nist_score.json", encoding="utf8") as f:
    r = json.load(f)

In [ ]:
for k in r:
    print(k,str(r[k]["avg_nist"]))